In [1]:
# load modules to view the bonds we need to move

In [69]:
from ase.io import read, write
from ase.visualize import view
from ase import Atoms
import os
import numpy as np
import random

In [31]:
frame = read('BaseFrame.xyz', ':')
baseframe = frame[0]
baseframe.wrap(eps=1e-15)

In [25]:
view(baseframe, viewer='ngl')

In [50]:
al_atoms = []
for i in range(len(baseframe)):
    if baseframe.symbols[i] == 'Al':
        al_atoms.append(i)

In [52]:
rand_al = al_atoms[random.randrange(len(al_atoms))]
print(rand_al)

104


In [110]:
# get near atoms
cell = [100, 100, 100]
positions_l = []
frame_distances = baseframe.get_all_distances()[rand_al]
for i in range(len(frame_distances)):
    if frame_distances[i] <= 3.8:
        symbol = baseframe.symbols[i]
        positions = baseframe.positions[i]
        positions_l.append(list(positions))
        print(f'{symbol} : {positions} : index {i}')
        
atoms = Atoms('AlO6', positions_l)

Al : [ 5.89594543 34.33632324 19.92149255] : index 104
O : [ 4.91328786 31.53952861 17.84468356] : index 113
O : [ 4.13283098 37.05752882 18.2453055 ] : index 121
O : [ 6.63671806 31.39590943 21.7356296 ] : index 122
O : [ 2.73632339 34.75773216 21.49374466] : index 127
O : [ 8.73620376 34.76340134 17.75397671] : index 246
O : [ 7.76677427 36.83265142 21.6052385 ] : index 260


In [74]:
view(atoms, viewer='ngl')

In [90]:
Alum = np.array([5.89594543, 34.33632324, 19.92149255])
Oxy = np.array([4.91328786, 31.53952861, 17.84468356])

origin = np.array([0, 0, 0])

vector_d = Oxy - Alum
vector_d 
#sum_d = 0
#for i in range(vector_d.shape[0]):
#    sum_d += float(vector_d[i])**2
    

array([-0.98265757, -2.79679463, -2.07680899])

In [105]:
b_OO = 3.6195
b_O = 3.6195
list_bs = []
array_BO = np.zeros([10,4])
for i in range(10):
    b_O -= 0.26195
    list_bs.append(b_O)
    vector_N = vector_d * (b_O/b_OO)
    print(f'{b_O}: {vector_N}')
    array_BO[i,0] = b_O
    array_BO[i,1] = vector_N[0]
    array_BO[i,2] = vector_N[1]
    array_BO[i,3] = vector_N[2]

array_BO = np.array(list_lines)
np.save('Al-O_vectors.npy', array_BO)

3.35755: [-0.9115408  -2.59438536 -1.92650643]
3.0955999999999997: [-0.84042403 -2.39197609 -1.77620387]
2.8336499999999996: [-0.76930726 -2.18956682 -1.62590131]
2.5716999999999994: [-0.69819049 -1.98715755 -1.47559875]
2.3097499999999993: [-0.62707372 -1.78474828 -1.32529619]
2.047799999999999: [-0.55595695 -1.58233901 -1.17499363]
1.785849999999999: [-0.48484018 -1.37992974 -1.02469107]
1.523899999999999: [-0.41372341 -1.17752047 -0.87438851]
1.2619499999999988: [-0.34260664 -0.9751112  -0.72408595]
0.9999999999999988: [-0.27148987 -0.77270193 -0.57378339]


In [109]:
# compute the new oxygen position vector as Cartesian coordinates
Oxy_array = np.zeros([10,3])
for i in range(array_BO.shape[0]):
    Oxy_array[i,:] = Alum + array_BO[i,1:]
    
Oxy_array
np.save('O_positions.npy', Oxy_array)

array([[ 4.98440463, 31.74193788, 17.99498612],
       [ 5.0555214 , 31.94434715, 18.14528868],
       [ 5.12663817, 32.14675642, 18.29559124],
       [ 5.19775494, 32.34916569, 18.4458938 ],
       [ 5.26887171, 32.55157496, 18.59619636],
       [ 5.33998848, 32.75398423, 18.74649892],
       [ 5.41110525, 32.9563935 , 18.89680148],
       [ 5.48222202, 33.15880277, 19.04710404],
       [ 5.55333879, 33.36121204, 19.1974066 ],
       [ 5.62445556, 33.56362131, 19.34770916]])

In [118]:
for i in range(Oxy_array.shape[0]):
    new_frame = baseframe
    positions = new_frame.positions
    positions[113] = Oxy_array[i,:]
    string = f'Al-O_{round(array_BO[i,0], 5)}.xyz'
    write(string, new_frame)